In [ ]:
import os
os.environ['PATH'] = '/usr/local/cuda-11.6/bin:' + os.environ['PATH']
if 'notebooks' not in os.listdir(os.getcwd()):
    os.chdir('../') #changing directories so that output/gsplat_full etc. exists

from contextlib import redirect_stdout
from video_api import initialize_gaussians
from gaussian_renderer import render
import sys 
import torch 
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

#assumes that the model and dataset are in the same directory as this notebook
sys.argv = ['']
gaussians, background_color, sample_cameras, kinematic_chain = initialize_gaussians(model_path='ur5')

def display_render(image_tensor: torch.Tensor):
    im = image_tensor.detach().permute(1, 2, 0).cpu().numpy()
    im = (im * 255).astype(np.uint8)
    plt.imshow(im)
    plt.axis('off')
    plt.show()

example_camera = sample_cameras[0]

In [ ]:
print(kinematic_chain.get_joint_parameter_names())
n = len(kinematic_chain.get_joint_parameter_names())    
example_camera.joint_pose = torch.tensor([0.6] * n).requires_grad_(True)
example_camera.world_view_transform.requires_grad_(True)

frame = torch.clamp(render(example_camera, gaussians, background_color)['render'], 0, 1)
print(frame.shape)
display_render(frame)

frame.sum().backward()
print('Sum of joint gradients:', example_camera.joint_pose.grad.sum())
print('Sum of camera gradients:', example_camera.world_view_transform.grad.sum())